In [ ]:
# front end:

import pandas as pd
from datetime import datetime as dt, date, time
import dash_bootstrap_components as dbc
from dash import Dash, html, dcc, State, Input, Output, dash_table, ctx
from plotly import graph_objects as go
from xbbg import blp
import sys
import os
sys.path.append(os.path.dirname(__file__))
from spread import Spread
from volume import Volume


# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

indices = {'FTSE 100': 'UKX INDEX', 'EURO STOXX 50': 'SX5E INDEX','DAX': 'DAX INDEX','CAC 40': 'CAC INDEX','IBEX 35': 'IBEX INDEX','SWISS MKT':'SMI INDEX'}


ln_colors = {'Black': '#000000',
          'White': '#ffffff',
          'Orange': '#ff8200',
          'Magenta': '#ec008b',
          'Fuchsia': '#b0008e',
          'Light Blue': '#00adec',
          'Teal': '#00c1b8',
          'Yellow': '#ffc23b',
          'Bright Yellow': '#edfc7c',
          'Purple': '#5f249f',
          'Navy': '#0b0c68',
          'Light Green': '#8eea5e',
          'Green': '#00b140',
          'Red': '#ee3728',
          'Medium Blue': '#0075ec',
          'Light Gray': '#b1c0c4',
          'Medium Gray': '#7c878e',
          'Dark Gray': '#2e3848',
          'Yellow Gray': '#c7c6bf',
          "Gold": "#FFC23B"
          }
default_bbg = ['VOD LN']

vc = Volume(default_bbg)
sc = Spread(default_bbg)


spread_mcap = go.Figure()
spread_value = go.Figure()

vol = go.Figure()
vol_n = go.Figure()

spread_mcap.add_trace(go.Scatter(x=[1], y=[1], name="Today"))
spread_mcap.add_trace(go.Scatter(x=(sc.historic_average()['time'] + dt(2023, 7, 27)).dt.time,y=sc.historic_average()['mcap_weightedSpread_bps'], name="20D Avg"))
spread_mcap.update_layout(
        title_text="Market Cap Weighted Spread",  # Update the title
        xaxis_title='Time',  # Update the x-axis label
        yaxis_title='Spread(bps)',width = 1300,height = 550)  # format tick labels as Hour:Minute



spread_value.add_trace(go.Scatter(x=[1], y=[1], name="Today"))
spread_value.add_trace(go.Scatter(x=(sc.historic_average()['time'] + dt(2023, 7, 27)).dt.time,y=sc.historic_average()['value_weightedSpread_bps'], name="20D Avg"))
spread_value.update_layout(
        title_text="Value Weighted Spread",  # Update the title
        xaxis_title='Time',  # Update the x-axis label
        yaxis_title='Spread(bps)',width = 1300,height = 550)


vol_n.add_trace(go.Scatter(x=[1], y=[1], name="Today"))
vol_n.add_trace( go.Scatter(x=(vc.historic_average()['time'] + dt(2023, 7, 27)).dt.time, y=vc.historic_average()['cum_vol$'], name="20D Avg"))
vol_n.update_layout(
    title_text="Notional Traded",  # Update the title
    xaxis_title='Time',  # Update the x-axis label
    yaxis_title='Notional Traded (USD)',width = 1300,height = 550)

vol.add_trace(go.Scatter(x=[1], y=[1], name="Today"))
vol.add_trace(go.Scatter(x=(vc.historic_average()['time'] + dt(2023, 7, 27)).dt.time, y=vc.historic_average()['cum_vol'], name="20D Avg"))
vol.update_layout(
    title_text="Volume",  # Update the title
    xaxis_title='Time',  # Update the x-axis label
    yaxis_title='Number of Shares Traded',width = 1300,height = 550)


app = Dash(__name__, update_title='Liquidity Metrics',external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    # dbc.Row(
    #     dbc.Col(html.H1("Liquidity Metrics", className='text-center text-primary mb-4'), width=20)
    # ),

    dbc.Row([
            dbc.Col([
                html.Img(
                    id='ln-logo',
                    src='assets/Liquidnet-Logo-FullColor.png',
                    style={
                        'height': '35px',
                        'width': 'auto',
                        'float': 'left',
                        'textAlign': 'left'
                    },
                ),
            ], width={"size": 2}),
            dbc.Col([
                html.H2(
                    children='LIQUIDITY METRICS',
                    style={'textAlign': 'right', 'fontSize': 24},
                    id='lis-market-share-label'
                ),
                html.Br(),
            ], width={"size": 4})
        ], justify="center", style={"marginTop": "30px"}),


    dbc.Row([
        dbc.Col([
            html.Div(
                [
                    dbc.Label("Enter your selected stocks' BBG tickers separated with commas "),
                    dbc.Input(id='data-set', placeholder="Bloomberg Tickers", type="text"),
                    dbc.Label("", id="status_code"),
                    # dbc.FormText("Type something in the box above"),
                    dbc.Button("Run", id="data-set-button")  # Added a button to trigger the callback
                ]
            ),
            html.Div(id='output-div')
        ], width={'size': 4, 'offset': 0, 'order': 1}),
    dbc.Col([
    html.Div([
        html.Label("Select a European Index"),  # Add this line
        dcc.Dropdown(
            id='my-dpdn2',
            multi=False,
            value=['FTSE 100'],
            options=[{'label': x, 'value': x} for x in ['FTSE 100', 'EURO STOXX 50', 'CAC 40','DAX','SWISS MKT','IBEX 35']],
            placeholder='European Indices'),
            dbc.Label("", id="status_code2"),
            dbc.Button("Run", id="data-set-button2")]

    ,style = {'marginTop': '8px'})
    ], width={'size': 4, 'offset': 0, 'order': 2}),

    dbc.Col([
        html.Div([
            html.Label("Select a Sector"),  # Add this line
            dcc.Dropdown(
                id='my-dpdn3',
                multi=False,
                value=['Financials'],
                options=[{'label': x, 'value': x} for x in
                         ['Materials', 'Utilities', 'Consumer Staples', 'Industrials', 'Financials', 'Consumer Discretionary', 'Real Estate', 'Health Care', 'Information Technology', 'Energy', 'Communication Services']],
                placeholder='Sector'),
            dbc.Label("", id="status_code3"),
            dbc.Button("Run", id="data-set-button3")]

            , style={'marginTop': '8px'}) ],
    width={'size': 4, 'offset': 0, 'order': 2}),

    ],justify='center'),
    dcc.Interval(interval=180000, id='interval'),
    dcc.Store(id='store-data', data=[], storage_type='memory'),

dbc.Row([
    dbc.Col(dcc.Graph(id='spread_mcap_chart', figure=spread_mcap), width=4),
    dbc.Col(dcc.Graph(id='spread_value_chart', figure=spread_value), width=4,),
],justify='center'),

dbc.Row([
    dbc.Col(dcc.Graph(id='avat_n_chart', figure=vol_n), width=4),
    dbc.Col(dcc.Graph(id='avat_chart', figure=vol), width=4),
],justify='center',style={"marginLeft": "30px"})

], fluid=True)



@app.callback(
    Output('store-data','data', allow_duplicate=True),
    Input('data-set-button', 'n_clicks'),
    State('data-set','value'),
    prevent_initial_call=True
)
def store_data_e(_, value):     #store_data for individual stocks
    '''Exception Handling string.split(',') VOD LN,TCAP LN Then return the clean data'''
    data = value.upper().split(', ')
    sc.update_name_list(data)
    vc.update_name_list(data)

    return data


@app.callback(
    Output('store-data','data', allow_duplicate=True),
    Input('data-set-button2', 'n_clicks'),
    State('my-dpdn2','value'),
    # allow_duplicate = True,
    prevent_initial_call=True
)
def store_data_i(_, value):     #store_data for indices
    '''Exception Handling string.split(',') VOD LN,TCAP LN Then return the clean data'''
    index_df = blp.bds(tickers = indices[value], flds = 'INDX_MEMBERS')
    index = index_df['member_ticker_and_exchange_code'].tolist()

    sc.update_name_list(index)
    vc.update_name_list(index)

    return index


@app.callback(
    Output('store-data','data', allow_duplicate=True),
    Input('data-set-button3', 'n_clicks'),
    State('my-dpdn3','value'),
    # allow_duplicate = True,
    prevent_initial_call=True
)
def store_data_s(_, value):     #store_data for sector
    '''Exception Handling string.split(',') VOD LN,TCAP LN Then return the clean data'''

    indices_bbg = blp.bds(tickers='UKX Index', flds='INDX_MEMBERS')
    stocks = indices_bbg['member_ticker_and_exchange_code'].tolist()

    names = list((x + ' Equity' for x in stocks))

    df = (blp.bdp(names, ['Security_name', 'GICS_Sector_Name']))

    sectors = df['gics_sector_name'].unique().tolist()

    sec_names = df[df['gics_sector_name'] == value].index.tolist()

    bbg_tickers = [item.replace(' Equity', "") for item in sec_names]

    sc.update_name_list(bbg_tickers)
    vc.update_name_list(bbg_tickers)

    return bbg_tickers




@app.callback(
    Output("spread_mcap_chart", "figure"),
    Output("spread_value_chart", "figure"),
    Output("avat_chart", "figure"),
    Output("avat_n_chart", "figure"),
    Output("status_code", "children"),

    Input("interval", "n_intervals"),
    Input('store-data','data'),

    # Input("spread_mcap", "n_intervals"),
    # Input("avat_n_chart", "n_intervals"),


    State("spread_mcap_chart", "figure"),
    State("spread_value_chart", "figure"),
    State("avat_chart", "figure"),
    State("avat_n_chart", "figure")

)
def get_updated_chart(_, names, spread_mcap_chart,spread_value_chart,avat_chart,avat_n_chart):
    '''Adjust format of names to feed into update_name_list'''
    # Code to update data in spread_mcap_chart and avat_chart graphs
    #lets first update the names:

    print(names)

    df = sc.intraday_updater()
    spread_mcap_chart["data"][0]["y"] = tuple(df["mcap_weightedSpread_bps"])
    spread_mcap_chart["data"][0]["x"] = tuple(df["date_time"].dt.time)

    spread_value_chart["data"][0]["y"] = tuple(df["value_weightedSpread_bps"])
    spread_value_chart["data"][0]["x"] = tuple(df["date_time"].dt.time)

    df2 = vc.intraday_updater()
    avat_n_chart["data"][0]["y"] = tuple(df2["cum_vol$"])
    avat_n_chart["data"][0]["x"] = tuple(df2["date_time"].dt.time)

    avat_chart["data"][0]["y"] = tuple(df2["cum_vol"])
    avat_chart["data"][0]["x"] = tuple(df2["date_time"].dt.time)

    hdf = sc.historic_average()     #historic
    hdf2 = vc.historic_average()

    spread_mcap_chart["data"][1]["y"] = tuple(hdf["mcap_weightedSpread_bps"])
    spread_mcap_chart["data"][1]["x"] = tuple((hdf["time"]+dt(2023, 7, 27)).dt.time)

    spread_value_chart["data"][1]["y"] = tuple(hdf["value_weightedSpread_bps"])
    spread_value_chart["data"][1]["x"] = tuple((hdf["time"]+dt(2023, 7, 27)).dt.time)

    avat_n_chart["data"][1]["y"] = tuple(hdf2["cum_vol$"])
    avat_n_chart["data"][1]["x"] = tuple((hdf2["time"]+dt(2023, 7, 27)).dt.time)

    avat_chart["data"][1]["y"] = tuple(hdf2["cum_vol"])
    avat_chart["data"][1]["x"] = tuple((hdf2["time"]+dt(2023, 7, 27)).dt.time)

    # return spread_mcap_chart
    print(names)
    return spread_mcap_chart,spread_value_chart,avat_chart,avat_n_chart, " Success "


if __name__ == '__main__':
    app.run_server(port=3020, debug=True)